In [3]:
%pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import wandb
import random

from collections import Counter
import math 

from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from nltk.translate.bleu_score import sentence_bleu

from tokenizers import Tokenizer
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup

In [5]:
!mkdir /kaggle/working/model

In [6]:
wandb.login(key="2448fa0c4f7d7dcf19f87fbb3ce5a9743515b380")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
wandb.init(
    
    project="G2P",
    config={
    "BATCH_SIZE":  2048,
    "NUM_EPOHS":  30,
    "LR": 3e-4,
    "D_MODEL":  256,
    "D_FF": 1024,
    "NUM": 3,
    "NUM_HEADS" : 4,
    "RANDOM_STATE": 42,
    "MAX_LEN": 32,
    "SAVE_DIR": "/kaggle/working/model",
    "BOS_TOKEN": "<bos>",
    "EOS_TOKEN": "<eos>",
    "PAD_TOKEN": "<pad>",
    'UNK_TOKEN': "<unk>"
    },
    name="bpe256_lr_3e-4_beam_bs2048_BigData_pretr",
)

wandb: Currently logged in as: nicitacom2018. Use `wandb login --relogin` to force relogin


In [9]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(wandb.config.RANDOM_STATE)

In [10]:
device  = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [11]:
path2 = "/kaggle/input/mtusiii/lexicon_librispeech-lexicon-200k-g2p.txt"
path3 = "/kaggle/input/mtusiii/my_cmu.txt"

with open(path2, "r") as file:
    text2 = file.readlines()
with open(path3, "r") as file:
    text3 = file.readlines()

In [12]:
class CreateDataset:
    def __init__(self, sentences):
        self.data = sentences
        
    def __getitem__(self, idx: int):
        self.tok_line = self.tokenize_line(self.data[idx])
        return self.tok_line

    def __len__(self):
        return len(self.data)
    
    def tokenize_line(self,seq):
        tok_line = seq.split()
        
        return {"input" :   tok_line[0],
                "label":  tok_line[1:]}

In [13]:
data = CreateDataset(text2+text3)

In [14]:
data_trg = ['Ġ'.join(list(i["label"])) for i in data ]
data_srs = [''.join(list(i["input"].upper())) for i in data ]

In [15]:
def decode(tokens):
    return ''.join(tokens).split('Ġ')

In [16]:
tokenizer_path = "/kaggle/input/bpeqqe/bpe.json"
tokenizer = Tokenizer.from_file(tokenizer_path)

In [17]:
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

In [18]:
class Dataset_WithPudding:
    def __init__(self, sentences , max_len: int = 32, debag=False):
        self.ds = sentences
        self.seq_len = max_len 
        self.debag = debag
        self.pad_id = torch.tensor(tokenizer.encode(wandb.config.PAD_TOKEN).ids)

    def __getitem__(self, idx: int):
        
        src_target_pair = self.ds[idx]
        
        src_text = ''.join(list(src_target_pair["input"].upper()))
        tgt_text = 'Ġ'.join(list(src_target_pair['label']))
        
        if self.debag:
            print(f"{src_text} -- src_text")
            print(f"{tgt_text} -- tgt_text")
            
            
        enc_input_tokens = tokenizer.encode(src_text).ids 
        dec_input_tokens = tokenizer.encode(tgt_text).ids 

    
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2
        # We will only add <s>, and </s> only on the label
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1
        
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens<0:
            print(f"{src_text} -- src_text")
            print(f"{decode(tgt_text)} -- tgt_text")
        
        #print(enc_num_padding_tokens , dec_num_padding_tokens)
        
        encoder_input = torch.cat(
            [
                torch.tensor(tokenizer.encode(wandb.config.BOS_TOKEN).ids),
                torch.tensor(enc_input_tokens),
                torch.tensor(tokenizer.encode(wandb.config.EOS_TOKEN).ids),
                self.pad_id.repeat(enc_num_padding_tokens)
            ],
            dim=0,
        )
        
        

        # Add only <s> token
        decoder_input = torch.cat(
            [
                torch.tensor(tokenizer.encode(wandb.config.BOS_TOKEN).ids),
                torch.tensor(dec_input_tokens),
                self.pad_id.repeat(dec_num_padding_tokens)
            ],
            dim=0,
        )

        # Add only </s> token
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor(tokenizer.encode(wandb.config.EOS_TOKEN).ids),
                self.pad_id.repeat(dec_num_padding_tokens),
            ],
            dim=0,
        )
        

        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len




        return {
            "encoder_input": encoder_input,  # (seq_len)
            "decoder_input": decoder_input,  # (seq_len)
            "encoder_mask": (encoder_input != self.pad_id).unsqueeze(0).unsqueeze(0).int(), # (1, 1, seq_len)
            "decoder_mask": (decoder_input != self.pad_id).unsqueeze(0).int() & causal_mask(decoder_input.size(0)), # (1, seq_len) & (1, seq_len, seq_len),
            "label": label,  # (seq_len)
            "src_text": src_text.upper(), # (seq_len)
            "tgt_text": ' '.join(decode(tgt_text)) # (seq_len)
        }


    def __len__(self) -> int:
        return len(self.ds)
    


In [19]:
def collate_fn(bath):
    encoder_input = torch.stack([i["encoder_input"].to(device) for i in bath])
    decoder_input =  torch.stack([i["decoder_input"].to(device) for i in bath])
    encoder_mask =  torch.stack([i["encoder_mask"].to(device) for i in bath])
    decoder_mask = torch.stack([i["decoder_mask"].to(device) for i in bath])
    label = torch.stack( [i["label"] for i in bath] )
    src_text = [i["src_text"] for i in bath]
    tgt_text = [i["tgt_text"] for i in bath]
    
    return {
            "encoder_input": encoder_input,  # (seq_len)
            "decoder_input": decoder_input,  # (seq_len)
            "encoder_mask": encoder_mask, # (1, 1, seq_len)
            "decoder_mask": decoder_mask, # (1, seq_len) & (1, seq_len, seq_len),
            "label": label,  # (seq_len)
            "src_text": src_text, # (seq_len)
            "tgt_text": tgt_text, # (seq_len)
        }

In [20]:
dataset = Dataset_WithPudding(data)

In [21]:
train_data , test_data = train_test_split( dataset ,test_size = 0.15)
val_data ,test_data = train_test_split(test_data ,test_size = 0.7)

In [24]:
batch_size = wandb.config.BATCH_SIZE
train_dataloader = DataLoader(
        train_data,
        shuffle=True,
        batch_size=batch_size,
        drop_last=True,
        collate_fn=collate_fn
                            )

val_dataloader = DataLoader(
        val_data,
        shuffle=True,
        batch_size=batch_size,
        drop_last=True,
        collate_fn=collate_fn
                            )

test_dataloader =DataLoader(
        test_data,
        shuffle=True,
        batch_size=batch_size,
        drop_last=True,
        collate_fn=collate_fn
                            )

In [29]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [30]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.depth = d_model // num_heads
        
        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)
        
        self.fc = nn.Linear(d_model, d_model)
        
    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)
    
    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)
        
        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)
        
        scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.depth, dtype=torch.float32))
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attn = F.softmax(scores, dim=-1)
        
        out = torch.matmul(attn, v)
        out = out.permute(0, 2, 1, 3).contiguous()
        out = out.view(batch_size, -1, self.d_model)
        
        out = self.fc(out)
        return out

In [31]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForwardNetwork, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [32]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadSelfAttention(d_model, num_heads)
        self.ffn = FeedForwardNetwork(d_model, d_ff, dropout)
        
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.layernorm1(x + self.dropout(attn_output))
        
        ffn_output = self.ffn(x)
        x = self.layernorm2(x + self.dropout(ffn_output))
        return x

In [33]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadSelfAttention(d_model, num_heads)
        self.cross_attn = MultiHeadSelfAttention(d_model, num_heads)
        self.ffn = FeedForwardNetwork(d_model, d_ff, dropout)
        
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.layernorm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        self_attn_output = self.self_attn(q=x, k=x, v=x, mask=tgt_mask)
        x = self.layernorm1(x + self.dropout(self_attn_output))
        
        cross_attn_output = self.cross_attn(q=x, k=enc_output, v=enc_output, mask=src_mask)
        x = self.layernorm2(x + self.dropout(cross_attn_output))
        
        ffn_output = self.ffn(x)
        x = self.layernorm3(x + self.dropout(ffn_output))
        return x

In [35]:
class TransformerBlock(nn.Module):
    def __init__(self, input_vocab_size, target_vocab_size, d_model=512, num_heads=8, num_encoder_layers=3, num_decoder_layers=3, d_ff=2048, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.encoder_embedding = nn.Embedding(input_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(target_vocab_size, d_model)
        
        self.pos_embedding = PositionalEncoding(d_model , wandb.config.MAX_LEN)
        
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_encoder_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_decoder_layers)])
        
        self.fc_out = nn.Linear(d_model, target_vocab_size)
    
    def encode(self, src, src_mask):
        src = self.pos_embedding(self.encoder_embedding(src))
        for layer in self.encoder_layers:
            src = layer(src, src_mask)
        return src
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        tgt = self.pos_embedding(self.decoder_embedding(tgt))
        for layer in self.decoder_layers:
            tgt = layer(tgt, memory, src_mask, tgt_mask)
        return tgt
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        memory = self.encode(src, src_mask)
        output = self.decode(memory, src_mask, tgt, tgt_mask)
        output = self.fc_out(output)
        return output

In [ ]:
def bleu_score(word,test_pronunciation):
    return sentence_bleu(word, test_pronunciation)

In [37]:
def phoneme_error_rate(predicted, target):
    m, n = len(predicted), len(target)
    if m == 0:
        return float(n)
    if n == 0:
        return float(m)

    d = np.zeros((m + 1, n + 1), dtype=int)

    for i in range(m + 1):
        d[i][0] = i
    for j in range(n + 1):
        d[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if predicted[i - 1] == target[j - 1] else 1
            d[i][j] = min(d[i - 1][j] + 1,    # Deletion
                          d[i][j - 1] + 1,    # Insertion
                          d[i - 1][j - 1] + cost)  # Substitution
            if i > 1 and j > 1 and predicted[i - 1] == target[j - 2] and predicted[i - 2] == target[j - 1]:
                d[i][j] = min(d[i][j], d[i - 2][j - 2] + cost)  # Transposition

    num_errors = d[m][n]
    num_phonemes = len(target)

    return num_errors / num_phonemes


In [38]:
def word_error_rate(predicted, target):
    return 0 if predicted == target else 1

In [133]:
def greedy_decode(model, src, src_mask, max_len, trg ,start_symbol , metricks:False):
    src = src.unsqueeze(0)  
    src_mask = src_mask.unsqueeze(0)  

    input_decoder = model.encode(src, src_mask)
    label = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)  

    for i in range(max_len - 1):
        tgt_mask = (torch.tril(torch.ones((label.size(1), label.size(1)))).type_as(src.data)).unsqueeze(0)
        out = model.decode(input_decoder, src_mask, label, tgt_mask)
        prob = model.fc_out(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        label = torch.cat([label, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == tokenizer.encode(wandb.config.EOS_TOKEN).ids[0]: 
            break
    if metricks:
        pred = decode(tokenizer.decode(label[0].tolist()))    
        per = phoneme_error_rate( pred  , trg.split() )
        wer = word_error_rate( pred  , trg.split() )
        bleu = bleu_score(pred  , trg.split())
    return label , per , wer ,bleu

In [134]:
def beam_search(model, src, src_mask, max_len, start_symbol, trg, metricks:False ,beam_size=5):
    src = src.unsqueeze(0)  
    src_mask = src_mask.unsqueeze(0) 
    memory = model.encode(src, src_mask)
    
    beams = [(torch.ones(1, 1).fill_(start_symbol).type_as(src.data), 0.0)]  # (последовательность, score)

    for _ in range(max_len - 1):
        new_beams = []
        for seq, score in beams:
            tgt_mask = causal_mask(seq.size(1)).unsqueeze(0).to(device)
            out = model.decode(memory, src_mask, seq, tgt_mask)
            prob = model.fc_out(out[:, -1])
            log_prob = F.log_softmax(prob, dim=-1)
            
            top_log_prob, top_indices = log_prob.topk(beam_size)

            for i in range(beam_size):
                next_seq = torch.cat([seq, torch.ones(1, 1).type_as(src.data).fill_(top_indices[0, i])], dim=1)
                new_score = score + top_log_prob[0, i].item()
                new_beams.append((next_seq, new_score))


        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
        
        if beams[0][0][0][-1].item() == tokenizer.encode(wandb.config.EOS_TOKEN).ids[0]:
            break
    
    best_seq, best_score = beams[0]
    
    if metricks:
        pred = decode(tokenizer.decode(best_seq[0].tolist()))    
        per = phoneme_error_rate( pred  , trg.split() )
        wer = word_error_rate( pred  , trg.split() )
        bleu = bleu_score(pred  , trg.split())
        
    return best_seq , per , wer ,bleu


In [152]:
def model_eval(model , dataloader , beam=True):
    model.eval()
    with torch.no_grad():
        per_list = []
        wer_list = []
        bleu_list = []
        for batch in tqdm( dataloader ):
            per_batch=0.0
            wer_batch=0.0
            bleu_batch=0.0
            for idx in range(len(batch)):
                encoder_input = batch['encoder_input'][idx].to(device)
                encoder_mask = batch['encoder_mask'][idx].to(device)
                trg = batch['tgt_text'][idx]
                if beam:
                    label ,per ,wer,bleu  = beam_search(
                                                 model=model,
                                                 src=encoder_input,
                                                 src_mask=encoder_mask,
                                                 max_len=wandb.config.MAX_LEN,
                                                 start_symbol=tokenizer.encode(wandb.config.BOS_TOKEN).ids[0],
                                                 trg=trg ,
                                                 metricks=True
                                                  )
                else:
                    label ,per ,wer ,bleu  = greedy_decode(
                                                 model=model,
                                                 src=encoder_input,
                                                 src_mask=encoder_mask,
                                                 max_len=wandb.config.MAX_LEN,
                                                 start_symbol=tokenizer.encode(wandb.config.BOS_TOKEN).ids[0],
                                                 trg=trg ,
                                                 metricks=True
                                                  )
                per_batch+=per
                wer_batch+=wer
                bleu_batch+=bleu

            per_list.append(per_batch / len(batch))
            wer_list.append(wer_batch / len(batch))
            bleu_list.append(bleu_batch / len(batch))

    return sum(per_list)/len(dataloader) , sum(wer_list)/len(dataloader),  sum(bleu_list)/len(dataloader)

In [61]:
best_per = 8

In [109]:
model = torch.load("/kaggle/input/g2p423/model5.62.pt")
optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.LR , betas=(0.9, 0.98))
num_epohs = wandb.config.NUM_EPOHS

num_training_steps = len(train_dataloader) * num_epohs
num_warmup_steps = int(0.1 * num_training_steps) 
loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer.encode(wandb.config.PAD_TOKEN).ids[0] , label_smoothing = 0.1 ).to(device)

In [63]:
num_cycles = 5
scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
                                                               optimizer=optimizer ,
                                                               num_warmup_steps=num_warmup_steps,
                                                               num_training_steps=num_training_steps,
                                                               num_cycles=num_cycles
                                                                )

In [155]:
model = torch.load("/kaggle/input/g2p423/model5.62.pt")
per , wer, bleu = model_eval(model , test_dataloader)
per*100 , wer*100 , bleu*100

  0%|          | 0/16 [00:00<?, ?it/s]

(7.937008428079857, 25.89285714285714, 83.59219769230634)

In [156]:
model_4 = torch.load("/kaggle/working/model/model0.049180807854277246.pt")
per , wer , bleu = model_eval(model_4 , test_dataloader)
per*100 , wer*100 ,bleu*100

  0%|          | 0/16 [00:00<?, ?it/s]

(8.797455025133596, 33.03571428571428, 84.10726656384213)

In [ ]:
for epoch in range(num_epohs):
    #train
    model.train()
    loss_epoch = 0.0
    for batch in tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}"):
        optimizer.zero_grad()
        
        encoder_input = batch['encoder_input'].to(device)# (B, seq_len)
        decoder_input = batch['decoder_input'].to(device) # (B, seq_len)
        encoder_mask = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
        decoder_mask = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)
        label = batch['label'].to(device)
        
        output = model(encoder_input, decoder_input ,encoder_mask ,decoder_mask)
        loss = loss_fn(output.view(-1 , tokenizer.get_vocab_size()), label.view(-1))
        loss_epoch += loss.item()
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    per , wer = model_eval(model , val_dataloader)

    if per < best_per:
        best_per = per 
        torch.save(model ,  f"/kaggle/working/model/model{per}.pt") 
    print(f"PER: {per*100} | loss: {loss_epoch/len( train_dataloader)} | WER: {wer*100} , lr:{scheduler.get_lr()}" )
    wandb.log({"PER": per*100, "loss": loss_epoch/len( train_dataloader), "WER": wer*100 }) 

In [158]:
with torch.no_grad():
    model.eval()
    for batch in tqdm( test_dataloader ):
        for idx in range(len(batch)):
            encoder_input = batch['encoder_input'][idx].to(device)
            encoder_mask = batch['encoder_mask'][idx].to(device)
            trg = batch['tgt_text'][idx]
            label ,per ,wer,bleu   = beam_search(model=model,
                                         src=encoder_input,
                                         src_mask=encoder_mask,
                                         max_len=wandb.config.MAX_LEN,
                                         start_symbol=tokenizer.encode(wandb.config.BOS_TOKEN).ids[0],
                                         trg=trg ,
                                         metricks=True
                                          )
                                                             
            print(f"{trg.split()} -- trg")
            print(f"{decode(tokenizer.decode(label.tolist()[0]))} -- pred")
            print(per)
            print(wer)
            print(bleu_score(trg.split() , decode(tokenizer.decode(label.tolist()[0]))))
            break

  0%|          | 0/16 [00:00<?, ?it/s]

['S', 'IH0', 'M', 'AE1', 'N', 'T', 'IH0', 'K'] -- trg
['S', 'IH0', 'M', 'AE1', 'N', 'T', 'IH0', 'K'] -- pred
0.0
0
0.8891397050194614
['EH0', 'R', 'OW1', 'N', 'IY2'] -- trg
['ER0', 'OW1', 'N', 'IY0'] -- pred
0.6
1
0.7071067811865476
['M', 'AH1', 'G', 'F', 'UH2', 'L'] -- trg
['M', 'AH1', 'G', 'F', 'UH2', 'L'] -- pred
0.0
0
0.9036020036098448
['D', 'IY1', 'M', 'AH0', 'M'] -- trg
['D', 'IY1', 'M', 'AH0', 'M'] -- pred
0.0
0
0.7952707287670506
['P', 'AE1', 'T', 'AH0', 'K', 'IY0'] -- trg
['P', 'AH0', 'T', 'EY1', 'K', 'IY0'] -- pred
0.3333333333333333
1
0.8408964152537145
['W', 'AO1', 'L', 'CH', 'ER0'] -- trg
['W', 'AO1', 'L', 'CH', 'ER0'] -- pred
0.0
0
0.7952707287670506
['F', 'OW1', 'K', 'AH0', 'S', 'IH0', 'NG'] -- trg
['F', 'OW1', 'K', 'AH0', 'S', 'IH0', 'NG'] -- pred
0.0
0
0.8091067115702212
['S', 'W', 'IH1', 'N', 'D', 'L', 'ER0', 'Z'] -- trg
['S', 'W', 'IH1', 'N', 'D', 'L', 'ER0', 'Z'] -- pred
0.0
0
0.9306048591020996
['P', 'OW1', 'K', 'S'] -- trg
['P', 'OW1', 'K', 'S'] -- pred
0.0
0
0.9

In [131]:
def bleu_score(word,test_pronunciation):
    return sentence_bleu(word, test_pronunciation)
